In [57]:
import pandas as pd
from pyproj import Transformer

In [58]:
def budget(comparison_result, non_repeated_multiplier=10000, repeated_multiplier=0, new_multiplier=50000): # $10k for moving cameras, $50k for new cameras

    reinstall = comparison_result["active_old_locations_count"] * non_repeated_multiplier
    no_reinstall = comparison_result["repeated_locations_count"] * repeated_multiplier
    new = (comparison_summary["new_locations_count"] - comparison_summary["active_old_locations_count"]) * new_multiplier

    total_cost = reinstall + no_reinstall + new
    return total_cost

In [59]:
def compare_camera_locations(oldlocations,
                             newlocations,
                             old_lat_col='lat',
                             old_lon_col='lon',
                             new_x_col='x_coord',
                             new_y_col='y_coord',
                             xy_epsg="EPSG:3857",
                             precision=6):

    df_old = pd.read_csv(oldlocations)
    df_new = pd.read_csv(newlocations)

    # filter active cameras
    if "status_clean" in df_old.columns:
        df_old = df_old[df_old["status_clean"].str.lower() == "active"]

    # convert to lat/long
    transformer = Transformer.from_crs(xy_epsg, "EPSG:4326", always_xy=True)
    df_new['lon'], df_new['lat'] = transformer.transform(df_new[new_x_col],
                                                         df_new[new_y_col])

    df_old['lat_r'] = df_old[old_lat_col].round(precision)
    df_old['lon_r'] = df_old[old_lon_col].round(precision)
    df_new['lat_r'] = df_new['lat'].round(precision)
    df_new['lon_r'] = df_new['lon'].round(precision)

    old_set = set(zip(df_old['lat_r'], df_old['lon_r']))
    new_set = set(zip(df_new['lat_r'], df_new['lon_r']))

    # compare coords
    same = old_set & new_set
    only_old = old_set - new_set
    only_new = new_set - old_set

    non_repeated_count = len(only_old) + len(only_new)
    repeated_count = len(same)

    return {
        "active_old_locations_count": len(df_old) + 1,
        "new_locations_count": len(df_new),
        "repeated_locations_count": repeated_count,
        "non_repeated_locations_count": non_repeated_count
    }

if __name__ == "__main__":
    comparison_summary = compare_camera_locations(
        "speed_cameras_clean (2).csv",
        "optimal_camera_sites_K150.csv",
        new_x_col="x_coord",
        new_y_col="y_coord",
        xy_epsg="EPSG:3857"
    )

    cost = budget(comparison_summary)

    print("Total Current Active Cameras:", comparison_summary["active_old_locations_count"])
    print("Total New Cameras:", comparison_summary["new_locations_count"])
    print("Repeated Locations:", comparison_summary["repeated_locations_count"])
    print("")
    print("Old Cameras to be Reinstalled:", comparison_summary["active_old_locations_count"])
    print("New Cameras to be Built:", comparison_summary["new_locations_count"] - comparison_summary["active_old_locations_count"])
    print("Final Cost for New Plan: $", cost)

Total Current Active Cameras: 150
Total New Cameras: 150
Repeated Locations: 0

Old Cameras to be Reinstalled: 150
New Cameras to be Built: 0
Final Cost for New Plan: $ 1500000


In [60]:
def compare_camera_locations(oldlocations,
                             newlocations,
                             old_lat_col='lat',
                             old_lon_col='lon',
                             new_x_col='x_coord',
                             new_y_col='y_coord',
                             xy_epsg="EPSG:3857",
                             precision=6):

    df_old = pd.read_csv(oldlocations)
    df_new = pd.read_csv(newlocations)

    # filter active cameras
    if "status_clean" in df_old.columns:
        df_old = df_old[df_old["status_clean"].str.lower() == "active"]

    # convert to lat/long
    transformer = Transformer.from_crs(xy_epsg, "EPSG:4326", always_xy=True)
    df_new['lon'], df_new['lat'] = transformer.transform(df_new[new_x_col],
                                                         df_new[new_y_col])

    df_old['lat_r'] = df_old[old_lat_col].round(precision)
    df_old['lon_r'] = df_old[old_lon_col].round(precision)
    df_new['lat_r'] = df_new['lat'].round(precision)
    df_new['lon_r'] = df_new['lon'].round(precision)

    old_set = set(zip(df_old['lat_r'], df_old['lon_r']))
    new_set = set(zip(df_new['lat_r'], df_new['lon_r']))

    # compare coords
    same = old_set & new_set
    only_old = old_set - new_set
    only_new = new_set - old_set

    non_repeated_count = len(only_old) + len(only_new)
    repeated_count = len(same)

    return {
        "active_old_locations_count": len(df_old) + 1,
        "new_locations_count": len(df_new),
        "repeated_locations_count": repeated_count,
        "non_repeated_locations_count": non_repeated_count
    }

if __name__ == "__main__":
    comparison_summary = compare_camera_locations(
        "speed_cameras_clean (2).csv",
        "optimal_camera_sites_K250_R500.csv",
        new_x_col="x_coord",
        new_y_col="y_coord",
        xy_epsg="EPSG:3857"
    )

    cost = budget(comparison_summary)

    print("Total Current Active Cameras:", comparison_summary["active_old_locations_count"])
    print("Total New Cameras:", comparison_summary["new_locations_count"])
    print("Repeated Locations:", comparison_summary["repeated_locations_count"])
    print("")
    print("Old Cameras to be Reinstalled:", comparison_summary["active_old_locations_count"])
    print("New Cameras to be Built:", comparison_summary["new_locations_count"] - comparison_summary["active_old_locations_count"])
    print("Final Cost for New Plan: $", cost)

Total Current Active Cameras: 150
Total New Cameras: 250
Repeated Locations: 0

Old Cameras to be Reinstalled: 150
New Cameras to be Built: 100
Final Cost for New Plan: $ 6500000
